In [1]:
import os
from app_functions import read_and_sort_data, apply_mask
import pandas as pd

In [4]:
def filter_data(pivoted_data_directory_filepath, min_stocks_per_date_ratio=0.0, min_total_dates_ratio=0.0, expected_stocks_per_date=1, mask=None):
    """
    Given a path to the directory with pivoted data files, filter the data based on the criteria below.

    :param pivoted_data_directory_filepath: (string) path to directory containing pivoted data
    :param min_stocks_per_date_ratio: (float) minimum ratio of stocks to expected stocks per date
    :param min_total_dates_ratio: (float) minimum ratio of total dates that meet the min_stocks_per_date_ratio criterion
    :param expected_stocks_per_date: (int) expected number of stocks per date
    :param mask: (dataframe) mask to apply to the data

    :return: good_dfs (dict), bad_dfs (dict)
    """

    good_dfs = {}
    bad_dfs = {}
    
    for filename in os.listdir(pivoted_data_directory_filepath):

        filepath = os.path.join(pivoted_data_directory_filepath,filename)
        if os.path.isfile(filepath):
            print(filename)
            # Read and sort the data file
            df = read_and_sort_data(filepath)

            # If expected_stocks_per_date is not passed in as an argument, set it as the number of columns in df
            if expected_stocks_per_date == 1:
                expected_stocks_per_date = df.shape[1]

            # Apply mask to the data, if provided
            masked_df = apply_mask(df,mask)
    
            # Drop rows and columns with all NaN values
            masked_df = masked_df.dropna(axis=0,how='all').dropna(axis=1,how='all')

            # Filter data based on the min_stocks_per_date_ratio and expected_stocks_per_date criteria
            masked_df = masked_df.loc[(masked_df.notna().sum(axis=1)/expected_stocks_per_date) > min_stocks_per_date_ratio]

            # Calculate ratio of dates that meet the above criterion
            maintained_dates_ratio = len(masked_df)/len(df)
            
            # Categorize data into good or bad based on the min_total_dates_ratio criterion
            if maintained_dates_ratio > min_total_dates_ratio:
                good_dfs[filename.split('.')[0]] = df.loc[masked_df.index]
            else:
                bad_dfs[f"{filename.split('.')[0]}: {maintained_dates_ratio}"] = df.loc[masked_df.index]
    
    return good_dfs,bad_dfs

In [5]:
n_quantiles = 10
min_stocks_per_date_ratio =0.8
min_total_dates_ratio = 0.8
data_directory_filepath = r'Z:\Interés Departamental\Model Portfolio\Hugo\Ranking DIP European Equities\copia 12-04-2023\data\vertical_dowload_files'
mask_filepath = r'Z:\Interés Departamental\Model Portfolio\Hugo\Ranking DIP European Equities\copia 12-04-2023\data\PriceClose_vertical\monthly_constituents_filter.csv'
expected_stocks_per_date = 600

In [7]:
mask = read_and_sort_data(mask_filepath)
good_dfs, bad_dfs = filter_data(data_directory_filepath,min_stocks_per_date_ratio,min_total_dates_ratio,expected_stocks_per_date,mask)

CompanyMarketCapitalization.csv
EV.csv
EVTOEBIT.csv
EVTOEBITDA.csv
EVTOOperatingCashFlow.csv
EVToSales.csv
HistPEG.csv
NetDebtToEBITDA.csv
NetDebtToEV.csv
PE.csv
PriceClose.csv
PriceToBVPerShare.csv
PriceToCFPerShare.csv
PriceToSalesPerShare.csv
PriceToTangBVPerShare.csv
TotalDebtToEBITDA.csv
TotalDebtToEV.csv


In [8]:
good_dfs.keys()

dict_keys(['EV', 'EVTOEBIT', 'EVTOEBITDA', 'EVTOOperatingCashFlow', 'EVToSales', 'PE', 'PriceClose', 'PriceToBVPerShare', 'PriceToCFPerShare', 'PriceToSalesPerShare', 'PriceToTangBVPerShare', 'TotalDebtToEBITDA', 'TotalDebtToEV'])